In [16]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, applications



**Load Metadata**

In [17]:
# Load Metadata
train_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')

# Preprocess Images
def preprocess_image(image_path, img_size=(224, 224)):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found or unable to read: {image_path}")
    image = cv2.resize(image, img_size)
    image = image / 255.0
    return image



/tmp/ipykernel_96/439643142.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')


**Image Batch Generate**

In [21]:

# Generator for batch processing
def image_batch_generator(metadata, batch_size=32):
    num_samples = len(metadata)
    while True:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = metadata.iloc[offset:offset+batch_size]
            images = []
            labels = []
            for _, row in batch_samples.iterrows():
                img_id = row['isic_id']
                img_path = os.path.join('/kaggle/input/isic-2024-challenge/train-image/image', f'{img_id}.jpg')
                if not os.path.exists(img_path):
                    print(f"Image not found: {img_path}")
                    continue
                try:
                    image = preprocess_image(img_path)
                    images.append(image)
                    labels.append(row['target'])
                except FileNotFoundError as e:
                    print(e)
                    continue
            yield np.array(images), np.array(labels)

# Example of how to use the generator
batch_size = 32
train_generator = image_batch_generator(train_metadata, batch_size=batch_size)


**Define the Model**

In [24]:

# Define your model (example with EfficientNetB0)
def build_model(input_shape):
    base_model = applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model
    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs, outputs)
    return model



In [25]:
input_shape = (224, 224, 3)
model = build_model(input_shape)
model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])


> Train the Model with the Generator

In [27]:

# Training the model with the generator
steps_per_epoch = len(train_metadata) // batch_size



In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=3,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True),
        tf.keras.callbacks.EarlyStopping(patience=3)
    ]
)



Epoch 1/3
12533/12533 ━━━━━━━━━━━━━━━━━━━━ 1817s 145ms/step - accuracy: 0.9990 - loss: 0.0086
Epoch 2/3
    1/12533 ━━━━━━━━━━━━━━━━━━━━ 23:36 113ms/step - accuracy: 1.0000 - loss: 0.0013

/opt/conda/lib/python3.10/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/opt/conda/lib/python3.10/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


12533/12533 ━━━━━━━━━━━━━━━━━━━━ 1561s 125ms/step - accuracy: 0.9990 - loss: 0.0079
Epoch 3/3
 9148/12533 ━━━━━━━━━━━━━━━━━━━━ 7:31 133ms/step - accuracy: 0.9990 - loss: 0.0078

In [ ]:
# Evaluate Model (this will need a separate generator for validation data)
 val_generator = image_batch_generator(val_metadata, batch_size=batch_size)
 #val_steps = len(val_metadata) // batch_size
 model.load_weights('best_model.keras')
 val_loss, val_acc = model.evaluate(val_generator, steps=val_steps)
 print(f'Validation Accuracy: {val_acc}')

In [ ]:
# Function to predict a single image
def predict_single_image(image_path, model, img_size=(224, 224)):
    image = preprocess_image(image_path, img_size)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    prediction = model.predict(image)
    predicted_prob = prediction[0][0]
    predicted_label = 1 if predicted_prob > 0.5 else 0
    return predicted_prob, predicted_label

# Load the trained model
model.load_weights('best_model.keras')

# Predict a single image
image_path = '/kaggle/input/isic-2024-challenge/train-image/image/ISIC_0015864.jpg'  # Replace with your image path
predicted_prob, predicted_label = predict_single_image(image_path, model)

print(f"Predicted Probability: {predicted_prob}")
print(f"Predicted Label: {predicted_label}")

In [ ]:
def plot_image_with_prediction(image_path, predicted_prob, predicted_label):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.title(f"Predicted: {'Malignant' if predicted_label == 1 else 'Benign'} ({predicted_prob:.2f})")
    plt.axis('off')
    plt.show()

plot_image_with_prediction(image_path, predicted_prob, predicted_label)